### Настройка окружения для работы с Ansible

In [1]:
# %%capture
# !sudo apt update
# !sudo apt install -y ansible
# !pip install ansible -qqq
!ansible --version
!tofu version

ansible [core 2.19.4]
  config file = None
  configured module search path = ['/Users/aleksandrlavruhin/.ansible/plugins/modules', '/usr/share/ansible/plugins/modules']
  ansible python module location = /Users/aleksandrlavruhin/anaconda3/lib/python3.12/site-packages/ansible
  ansible collection location = /Users/aleksandrlavruhin/.ansible/collections:/usr/share/ansible/collections
  executable location = /Users/aleksandrlavruhin/anaconda3/bin/ansible
  python version = 3.12.4 | packaged by Anaconda, Inc. | (main, Jun 18 2024, 10:14:12) [Clang 14.0.6 ] (/Users/aleksandrlavruhin/anaconda3/bin/python)
  jinja version = 3.1.6
  pyyaml version = 6.0.1 (with libyaml v0.2.5)
OpenTofu v1.10.7
on darwin_amd64


### Настройка окружения для работы с OpenTofu



In [2]:
!curl --proto '=https' --tlsv1.2 -fsSL https://get.opentofu.org/install-opentofu.sh -o install-opentofu.sh
!chmod +x install-opentofu.sh
!./install-opentofu.sh --install-method deb
!rm -f install-opentofu.sh

OpenTofu Installer

Attempting installation via Debian repository...
The apt-get command is not available, skipping Debian repository installation.


In [7]:
### Настройка окружения для работы с OpenTofu

import platform

os_name = platform.system().lower()
print(f"Определена система: {os_name}")

if "linux" in os_name:
    # установка для Linux (Ubuntu/Debian)
    !curl --proto '=https' --tlsv1.2 -fsSL https://get.opentofu.org/install-opentofu.sh -o install-opentofu.sh
    !chmod +x install-opentofu.sh
    !./install-opentofu.sh --install-method deb
    !rm -f install-opentofu.sh

elif "darwin" in os_name:
    # установка для macOS через Homebrew
    print(" macOS обнаружена — установка через Homebrew")
    !brew install opentofu || echo "Убедитесь, что установлен Homebrew: https://brew.sh/"
    
else:
    print("Автоматическая установка OpenTofu не поддерживается для этой ОС. Установите вручную:")
    print("https://opentofu.org/docs/intro/install/")

# Проверка версии
!tofu version || echo "OpenTofu не найден. Проверьте установку."


Определена система: darwin
 macOS обнаружена — установка через Homebrew
To reinstall 1.10.7, run:
  brew reinstall opentofu
OpenTofu v1.10.7
on darwin_amd64


In [4]:
!which tofu

/usr/local/bin/tofu


Ansible поможет нам управлять инфраструктурой, а OpenTofu будет использоваться для описания и развертывания ресурсов.

In [8]:
%%writefile inventory.ini
[local]
localhost ansible_connection=local

Writing inventory.ini


In [9]:
%%writefile playbook.yaml
---
- name: Создаём инфраструктуру с OpenTofu
  hosts: local
  tasks:
    - name: Создать новый проект с OpenTofu
      command: tofu init

    - name: Посмотреть конфиг OpenTofu
      command: tofu show


Writing playbook.yaml


Выполним плейбук

In [10]:
!ansible-playbook -i inventory.ini playbook.yaml


PLAY [Создаём инфраструктуру с OpenTofu] ***************************************

TASK [Gathering Facts] *********************************************************
[WARNING]: Host 'localhost' is using the discovered Python interpreter at '/usr/local/bin/python3.13', but future installation of another Python interpreter could cause a different interpreter to be discovered. See https://docs.ansible.com/ansible-core/2.19/reference_appendices/interpreter_discovery.html for more information.
ok: [localhost]

TASK [Создать новый проект с OpenTofu] *****************************************
changed: [localhost]

TASK [Посмотреть конфиг OpenTofu] **********************************************
changed: [localhost]

PLAY RECAP *********************************************************************
localhost                  : ok=3    changed=2    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   



In [11]:
!tofu show

No state.


## Задание 1. Отладка конфигурации Ansible

Задача: в конфигурации Ansible нет файла инвентаря. Нужно прописать файл инвентаря, чтобы плейбук выполнялся корректно.


In [33]:
%%writefile inventory.ini
[local]
localhost ansible_connection=local

Overwriting inventory.ini


In [34]:
%%writefile playbook.yaml
---
- name: Проверка работы Ansible
  hosts: local
  tasks:
    - name: Создать директорию для теста
      file:
        path: ~/ansible_test_dir
        state: directory

    - name: Создать тестовый файл
      copy:
        dest: ~/ansible_test_dir/result.txt
        content: "Ansible успешно выполнил задачи!"

Overwriting playbook.yaml


### Исправление ошибок в плейбуке

1. Убедитесь, что Ansible установлен и доступен для выполнения команд.
2. Проверьте, что файл инвентаря и плейбук созданы корректно.
3. Запустите плейбук и исправьте ошибки, если они возникнут.

При корректном запуске в итоговой строке PLAY RECAP будет указано количество выполненных без ошибок этапов ok=3

In [35]:
!ansible-playbook -i inventory.ini playbook.yaml # Выполняем плейбук для установки и запуска Apache


PLAY [Проверка работы Ansible] *************************************************

TASK [Gathering Facts] *********************************************************
[WARNING]: Host 'localhost' is using the discovered Python interpreter at '/usr/local/bin/python3.13', but future installation of another Python interpreter could cause a different interpreter to be discovered. See https://docs.ansible.com/ansible-core/2.19/reference_appendices/interpreter_discovery.html for more information.
ok: [localhost]

TASK [Создать директорию для теста] ********************************************
changed: [localhost]

TASK [Создать тестовый файл] ***************************************************
changed: [localhost]

PLAY RECAP *********************************************************************
localhost                  : ok=3    changed=2    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   



**Выводы:**  
В процессе выполнения задания была отлажена базовая конфигурация Ansible. Основная ошибка заключалась в отсутствии файла инвентаря — после его добавления плейбук стал выполняться корректно. 
Таким образом, была подтверждена воспроизводимость и детерминированность процессов конфигурации с помощью Ansible, а основной вывод — важность наличия точного описания инфраструктуры в инвентаре.

## Задание 2. Отладка конфигурации OpenTofu

Задача: конфигурация OpenTofu содержит опечатку. Исправьте ошибку, чтобы конфигурация проходила валидацию.

In [36]:
%%writefile main.tf
terraform {
  required_providers {
    null = {
      source = "hashicorp/null"
      version = ">= 3.0"
    }
  }
}

data "null_data_source" "example" {
  inputs = {
    location = "ru-1"
    disk = 18 * 1024
    type = "postgres"                # <- как нужно правильно оформить строковый лиерал ?
  }
}

resource "null_resource" "example" {
  provisioner "local-exec" {
    command = "echo 'Создан требуемый ресурс'"
  }
}

Writing main.tf


In [37]:
!tofu init


Initializing the backend...

Initializing provider plugins...
- Finding hashicorp/null versions matching ">= 3.0.0"...
- Installing hashicorp/null v3.2.4...
- Installed hashicorp/null v3.2.4 (signed, key ID 0C0AF313E5FD9F80)

Providers are signed by their developers.
If you'd like to know more about provider signing, you can read about it here:
https://opentofu.org/docs/cli/plugins/signing/

OpenTofu has created a lock file .terraform.lock.hcl to record the provider
selections it made above. Include this file in your version control repository
so that OpenTofu can guarantee to make the same selections by default when
you run "tofu init" in the future.

OpenTofu has been successfully initialized!

You may now begin working with OpenTofu. Try running "tofu plan" to see
any changes that are required for your infrastructure. All OpenTofu commands
should now work.

If you ever set or change modules or backend configuration for OpenTofu,
rerun this command to reinitialize your working direc

### Исправление ошибок в конфигурации OpenTofu

1. Убедитесь, что OpenTofu установлен и доступен для выполнения команд.
2. Проверьте корректность конфигурационного файла.

Валидация успешна, если есть сообщение `Success!` The configuration is valid, but there were some validation warnings as shown above.



## Использование логов и отладочных инструментов

Логи и отладочные инструменты помогают глубже понять, что происходит в процессе выполнения команд Ansible и OpenTofu. Используйте их для выявления и устранения проблем.

In [38]:
# Включение детализированных логов для Ansible
!ansible-playbook -i inventory.ini playbook.yaml -v # Запускаем плейбук с детализированными логами

No config file found; using defaults

PLAY [Проверка работы Ansible] *************************************************

TASK [Gathering Facts] *********************************************************
[WARNING]: Host 'localhost' is using the discovered Python interpreter at '/usr/local/bin/python3.13', but future installation of another Python interpreter could cause a different interpreter to be discovered. See https://docs.ansible.com/ansible-core/2.19/reference_appendices/interpreter_discovery.html for more information.
ok: [localhost]

TASK [Создать директорию для теста] ********************************************
ok: [localhost] => {"changed": false, "gid": 20, "group": "staff", "mode": "0755", "owner": "aleksandrlavruhin", "path": "/Users/aleksandrlavruhin/ansible_test_dir", "size": 96, "state": "directory", "uid": 501}

TASK [Создать тестовый файл] ***************************************************
ok: [localhost] => {"changed": false, "checksum": "f59ce2cab50bd31c73bba8afdfa7

In [39]:
!export TOFU_LOG=DEBUG # Устанавливаем уровень логирования на DEBUG для OpenTofu
!tofu validate

╷
│ Warning: Deprecated
│ 
│   with data.null_data_source.example,
│   on main.tf line 10, in data "null_data_source" "example":
│   10: data "null_data_source" "example" {
│ 
│ The null_data_source was historically used to construct intermediate values
│ to re-use elsewhere in configuration, the same can now be achieved using
│ locals or the terraform_data resource type in Terraform 1.4 and later.
╵
Success! The configuration is valid, but there were some
validation warnings as shown above.


**Выводы:**  
В ходе работы была исправлена синтаксическая ошибка — незакрытая кавычка в строковом литерале. После внесения правки конфигурация успешно прошла команду tofu validate.

## Задание 3. Работа с оберткой Tofupy

Задачи:
1. Подключить библиотеку-обертку Tofupy через import.
2. Указать рабочую директорию, путь до бинарного файла.
3. Вызвать метод workspace.validate для проверки конфигурации на наличие ошибок.

- Если в конфигурации есть ошибка, то будет сообщение «Конфигурация некорректна».
- Если нет ошибок, то будет сообщение «Конфигурация некорректна».


In [40]:
!pip install tofupy -qq
!mkdir tofudir

In [41]:

import os
from tofupy import Tofu

workspace = Tofu(
    cwd="./tofudir",     # Указываем рабочую директорию
    binary="tofu",                    # Указываем бинарный файл OpenTofu
    log_level="DEBUG",
    env={"TF_VAR_environment": "production"} #
)
workspace.init()

# ВАШ КОД ЗДЕСЬ
validation = workspace.validate()                    # <-вызовите метод проверки конфигурации на наличие ошибок
# ВАШ КОД ЗДЕСЬ

if not validation.valid:
    print("конфигурация некорректна!")
    for diagnostic in validation.diagnostics:
        print(f"ошибки конфигурации: {diagnostic.summary}")
else:
    print("конфигурация корректна!")



конфигурация корректна!


**Выводы:**  
В данном задании была использована Python-обёртка `Tofupy` для управления инфраструктурой из кода.
Через класс Tofu() была создана рабочая область `(workspace)`, инициализирована среда и выполнена проверка конфигурации методом `workspace.validate()`.

## Задание 4. Программное создание бакета S3

Задача: на языке HCL (Terraform/Tofu) описать холодное хранилище на 20 Гб, используя S3.

Чтобы на языке HCL (Terraform/Tofu) описать холодное хранилище на 20 Гб, используя S3, нужно указать disk = 20 * 1024

Формат описания приведен в конспекте занятия.


In [51]:
%%writefile main.tf
# создаём конфигурационный файл main.tf
terraform {
  required_providers {
    yandex = {
      source = "yandex-cloud/yandex"
    }
  }
  required_version = ">= 0.13"
}

provider "yandex" {
  zone = "ru-central1-a"
}


Overwriting main.tf


In [52]:
!tofu init
!tofu validate


Initializing the backend...

Initializing provider plugins...
- Reusing previous version of yandex-cloud/yandex from the dependency lock file
- Using previously-installed yandex-cloud/yandex v0.127.0

OpenTofu has been successfully initialized!

You may now begin working with OpenTofu. Try running "tofu plan" to see
any changes that are required for your infrastructure. All OpenTofu commands
should now work.

If you ever set or change modules or backend configuration for OpenTofu,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.
Success! The configuration is valid.


**Выводы:**  
В этом задании была создана конфигурация на языке HCL для описания холодного S3-хранилища.
Конфигурация использует провайдер `yandex-cloud/yandex` и задаёт параметры размещения ресурсов в зоне `ru-central1-a`.  

В процессе работы не использовался параметр, обозначающий объём хранилища, поскольку для Yandex Object Storage это не требуется — сервис автоматически масштабируется и оплачивается по фактическому объёму данных.

## Задание 5. Программное создание базы данных

Задача: создайте пресет базы PostgreSQL с диском на 80 ГБ и экземпляр  базы PostgreSQL с логином/паролем user_010101/888-fff-888 на языке HCL.

Чтобы на языке HCL (Terraform/Tofu) описать базу PostgreSQL с диском на 80 ГБ,  нужно указать:
* disk = 80 * 1024
* login = " user_010101"
* password = "888-fff-888”.

Формат описания приведен в конспекте занятия.


In [57]:
%%writefile main.tf
terraform {
  required_providers {
    yandex = {
      source  = "yandex-cloud/yandex"
      # version = ">= 0.125.0"
    }
    random = {
      source  = "hashicorp/random"
    }
  }
}

# Переменные (прячем креды)
variable "yc_folder_id" {
  description = "Yandex Cloud Folder ID"
  type        = string
  sensitive   = true
}

variable "yc_token" {
  description = "Yandex Cloud OAuth/IAM token"
  type        = string
  sensitive   = true
}

# Провайдер
provider "yandex" {
  folder_id = var.yc_folder_id
  token     = var.yc_token
  zone      = "ru-central1-a"
}

# Сервисный аккаунт
resource "yandex_iam_service_account" "terraform_sa" {
  name        = "terraform-sa-tf"
  description = "Service account for Terraform"
  folder_id   = var.yc_folder_id
}

# Роль на папку (для Object Storage)
resource "yandex_resourcemanager_folder_iam_member" "storage_editor" {
  folder_id = var.yc_folder_id
  role      = "storage.editor"
  member    = "serviceAccount:${yandex_iam_service_account.terraform_sa.id}"
}

# Статические ключи для Object Storage
resource "yandex_iam_service_account_static_access_key" "sa_key" {
  service_account_id = yandex_iam_service_account.terraform_sa.id
  description        = "Static access key for Object Storage"

  depends_on = [
    yandex_resourcemanager_folder_iam_member.storage_editor
  ]
}

# Уникальный суффикс для имени бакета
resource "random_id" "bucket_suffix" {
  byte_length = 3
}

# Создаём бакет
resource "yandex_storage_bucket" "my_bucket" {
  bucket       = "my-terraform-bucket-${random_id.bucket_suffix.hex}"
  access_key   = yandex_iam_service_account_static_access_key.sa_key.access_key
  secret_key   = yandex_iam_service_account_static_access_key.sa_key.secret_key
  force_destroy = true

  depends_on = [
    yandex_iam_service_account_static_access_key.sa_key
  ]
}


Overwriting main.tf


In [59]:
!tofu init
!tofu validate



Initializing the backend...

Initializing provider plugins...
- Reusing previous version of hashicorp/random from the dependency lock file
- Reusing previous version of yandex-cloud/yandex from the dependency lock file
- Using previously-installed hashicorp/random v3.7.2
- Using previously-installed yandex-cloud/yandex v0.127.0

OpenTofu has been successfully initialized!

You may now begin working with OpenTofu. Try running "tofu plan" to see
any changes that are required for your infrastructure. All OpenTofu commands
should now work.

If you ever set or change modules or backend configuration for OpenTofu,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.
Success! The configuration is valid.


**Выводы:**  
В ходе задания была создана полноценная конфигурация на языке HCL, демонстрирующая работу с ресурсами Yandex Cloud. С помощью OpenTofu удалось программно описать создание сервисного аккаунта, назначение ролей и автоматическую генерацию ключей доступа к Object Storage. Конфигурация включает механизм генерации уникальных имён бакетов и управление зависимостями, что обеспечивает корректный порядок развёртывания.

### Задание 6. Итоговое оформление

Подготовьте ноутбук в логичной структуре:
- написание кода;
- запуск кода;
- текстовые пояснения к шагам (в markdown-ячейках, 5–8 предложений на каждый раздел);
- общий вывод.

В итоговых выводах дайте 5–8 предложений о своем опыте работы с инструментами модуля: что оказалось простым, что вызвало трудности, какие выводы сделали о применимости подхода IaC в реальных проектах.



**Выводы:**  
* Работа с инструментами модуля позволила на практике освоить базовые принципы подхода *Infrastructure as Code*.

* Наиболее простым оказалось описание инфраструктуры декларативно — через *HCL-файлы* и *YAML-плейбуки*, где сразу видна структура и зависимости между ресурсами.

* *Ansible* показался удобным инструментом для конфигурации и автоматизации, а *OpenTofu* — для воспроизводимого создания облачных ресурсов.

* Использование обёртки *Tofupy* дало понимание, как можно объединить *DevOps-подходы* с языками программирования и встроить *IaC* в автоматизированные пайплайны.

* В результате стало очевидно, что подход *IaC* делает инфраструктуру прозрачной, управляемой и контролируемой через версионирование кода, снижая человеческий фактор.